In [1]:
import sys
sys.path.append("../")
from pyproj import CRS
import numpy as np
import sys
from osgeo import gdal
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from time import time
from collections import Counter
import pandas as pd #for data analysing
import openpyxl
import os



#### Télécharger les rasters 

In [3]:
###### monthly proba fields ###############
mask_mai_b = xr.open_dataset('December_b.nc')


In [4]:
###### monthly proba fields ###############
mask_mai_a = xr.open_dataset('December_a.nc')


In [6]:
mask = xr.merge([mask_mai_a, mask_mai_b], join='outer')

In [4]:
###### monthly proba fields ###############
mask_march = xr.open_dataset('July_b.nc')
mask_march

<xarray.Dataset>
Dimensions:    (longitude: 7117, latitude: 6200)
Coordinates:
  * longitude  (longitude) float64 8.125 8.126 8.126 8.126 ... 10.55 10.55 10.55
  * latitude   (latitude) float64 47.85 47.85 47.85 47.85 ... 45.74 45.74 45.74
Data variables:
    crs        int32 ...
    layer      (latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-03-25 16:20:40

In [5]:
###### monthly proba fields ###############
mask_april = xr.open_dataset('August_b.nc')
mask_april

<xarray.Dataset>
Dimensions:    (longitude: 7117, latitude: 6200)
Coordinates:
  * longitude  (longitude) float64 8.125 8.126 8.126 8.126 ... 10.55 10.55 10.55
  * latitude   (latitude) float64 47.85 47.85 47.85 47.85 ... 45.74 45.74 45.74
Data variables:
    crs        int32 ...
    layer      (latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-03-25 16:20:45

In [7]:
############ DEM sans classe ######################
DEM_b = xr.open_dataset('DEM_b.nc')

In [8]:
############ DEM sans classe ######################
DEM_a = xr.open_dataset('DEM_a.nc')

In [9]:
DEM = xr.merge([DEM_a, DEM_b], join='outer')

In [10]:
############ snow product provenant de summermask #######################
NDSI_MAM_b = xr.open_dataset('NDSI_DJF_summermask_b_nc.nc')
#NDSI_MAM = NDSI_MAM.drop_vars(['snow_final', 'snow_final2','Summer_mask'])

In [11]:
NDSI_MAM_a = xr.open_dataset('NDSI_DJF_summermask_a_nc.nc')

In [16]:
NDSI = xr.merge([NDSI_MAM_a, NDSI_MAM_b], join='outer')

#### Prendre unqiuement les mois qui nous intéressent

In [11]:
NDSI_MAM['z'] = pd.to_datetime(NDSI_MAM['z'].values)

In [12]:
######### sélectionner unqiuement les mois qui nous intéresse #############
NDSI_mai = NDSI_MAM.sel(z=NDSI_MAM['z.month'] == 6, drop=True)

NDSI_mai

<xarray.Dataset>
Dimensions:      (longitude: 7117, latitude: 6200, z: 36)
Coordinates:
  * longitude    (longitude) float64 8.125 8.126 8.126 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) datetime64[ns] 1985-06-01 1986-06-01 ... 2021-06-01
Data variables:
    DEM          (latitude, longitude) float32 ...
    snow_final2  (z, latitude, longitude) float32 ...
    crs          int32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-26)
    date:         2024-01-25 13:54:46
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

In [14]:
######### sélectionner unqiuement les mois qui nous intéresse #############
NDSI_march = NDSI_MAM.sel(z=NDSI_MAM['z.month'] == 7, drop=True)

NDSI_march

<xarray.Dataset>
Dimensions:      (longitude: 7117, latitude: 6200, z: 36)
Coordinates:
  * longitude    (longitude) float64 8.125 8.126 8.126 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) datetime64[ns] 1985-07-01 1986-07-01 ... 2021-07-01
Data variables:
    DEM          (latitude, longitude) float32 ...
    snow_final2  (z, latitude, longitude) float32 ...
    crs          int32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-26)
    date:         2024-01-25 13:54:46
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

In [16]:
######### sélectionner unqiuement les mois qui nous intéresse #############
NDSI_april = NDSI_MAM.sel(z=NDSI_MAM['z.month'] == 8, drop=True)

NDSI_april

<xarray.Dataset>
Dimensions:      (longitude: 7117, latitude: 6200, z: 36)
Coordinates:
  * longitude    (longitude) float64 8.125 8.126 8.126 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) datetime64[ns] 1985-08-01 1986-08-01 ... 2021-08-01
Data variables:
    DEM          (latitude, longitude) float32 ...
    snow_final2  (z, latitude, longitude) float32 ...
    crs          int32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-26)
    date:         2024-01-25 13:54:46
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

#### Fonctions à définir

In [18]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_cloudpercent_b(array):
    cloud = np.sum(array == 2)
    #print(cloud)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    percentage_cloud_b = round(((float(cloud) / float(tot_pix)) * 100),2)
    return percentage_cloud_b

In [19]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_cloudpercent_a(array_copy):
    cloud = np.sum(array_copy  == 2)
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    cloud_percent = round((float(cloud) / float(tot_pix)) * 100, 2)
    return cloud_percent

In [20]:
def step2_gapfilling_tot(array,array_copy):
    cloudpercent_b = compute_cloudpercent_b(array)
    cloudpercent_a = compute_cloudpercent_a(array_copy)
    step2_gf_tot = round(((cloudpercent_b)-(cloudpercent_a)),2)
    #step3_gf_tot = ((cloudpercent_b)-(cloudpercent_a))
    return step2_gf_tot

In [21]:
def compute_snowpercent_b(array):
    snow = np.count_nonzero((array == 1) | (array == 100))
    #print(snow)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    snow_percent_b = round(((float(snow) / float(tot_pix)) * 100),2)
    return snow_percent_b

In [22]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_snowpercent_a(array_copy):
    snow = np.count_nonzero((array_copy == 1) | (array_copy == 100))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    snow_percent = round((float(snow) / float(tot_pix)) * 100, 2)
    return snow_percent

In [23]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_landpercent_b(array):
    land = np.count_nonzero((array == 0) | (array == 200))
    #print(land)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    land_percent_b = round((float(land) / float(tot_pix)) * 100,2)
    return land_percent_b

In [24]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_landpercent_a(array_copy):
    land = np.count_nonzero((array_copy == 0) | (array_copy == 200))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    land_percent = round((float(land) / float(tot_pix)) * 100, 2)
    return land_percent

#### Transformer notre raster selon les conditions

In [50]:
NDSI_MAM_b['snow_final4'] = NDSI_MAM_b.copy()


HSmin500 = (np.nanmin(DEM_b.layer.isel(z=0).values[mask_mai_b.layer.values == 1]) + 500)
    # Créer un masque pour les pixels à modifier
to_modify = (NDSI_MAM_b['snow_final4'].values == 2) & (DEM_b.layer.isel(z=0).values > HSmin500) & (mask_mai_b.layer.values == 1)
# Modifier les valeurs du raster NDSI selon les conditions
NDSI_MAM_b['snow_final4'].values[to_modify] = 100

In [28]:
################ LAND ########################

#### On va remettre les rasters ensemble pour faire des ratsers de saison

In [32]:
# Fusionner les rasters en un seul en utilisant xr.merge()
NDSI_MAM = xr.merge([NDSI_mai, NDSI_march, NDSI_april])
NDSI_MAM = NDSI_MAM.sortby("z")

### On va calculer les pourcentage de changements

In [32]:
cloud_percent_b_list = []
cloud_percent_a_list = []
step2_gapfilling_tot_list = []
snow_percent_b_list = []
snow_percent_a_list = []
land_percent_b_list = []
land_percent_a_list = []
time_list = []

In [35]:
data = {
    'Time': time_list,
    'Cloud_Percent_Before': cloud_percent_b_list,
    'Cloud_Percent_After': cloud_percent_a_list,
    'Cloud_Reduction': step2_gapfilling_tot_list,
    'Snow_Percent_Before': snow_percent_b_list,
    'Snow_Percent_After': snow_percent_a_list,
    'Land_Percent_Before': land_percent_b_list,
    'Land_Percent_After': land_percent_a_list
}

# Créer un DataFrame à partir du dictionnaire
df_step4 = pd.DataFrame(data)
filename = f'df_monthlyproba_JJA_b.xlsx'
filepath = os.path.join('/home/charlottepoussin/Excels', filename)
df_step4.to_excel(filepath, index=False)